In [ ]:
import pystac_client
from odc.geo import BoundingBox

In [ ]:
client = pystac_client.Client.open("http://127.0.0.1:9588")
client

In [ ]:
import folium
from folium.plugins.timestamped_geo_json import TimestampedGeoJson
from tlz.functoolz import curry

In [ ]:
categories = {
    "bo": "#008888",
    "db": "#880000",
    "ml": "#880088",
    "mo": "#888800",
    "pf": "#000088",
    "rf": "#008800",
    "tg": "#ff88ff",
    "ts": "#81d4fa",
}


def style_function(_, category):
    fill_color = categories.get(category, "#000000")
    style = {
        # "fillColor": fill_color,
        "color": fill_color,
    }

    return {k: v for k, v in style.items() if v is not None}

In [ ]:
def extract_category(id):
    return id.split("-")[-1].lower()

In [ ]:
from tlz.itertoolz import groupby

In [ ]:
def as_geojson(items, style_function):
    for item in items:
        category = extract_category(item.collection_id)

        marker = folium.CircleMarker()
        yield folium.GeoJson(
            item.geometry,
            style_function=curry(style_function, category=category),
            marker=marker,
        )


def as_timestamped_geojson(items, style_function):
    def as_timestamped(item):
        category = extract_category(item.collection_id)

        style = style_function(None, category)
        return {
            "type": "Feature",
            "geometry": item.geometry,
            "properties": {
                "times": item.properties["time"],
                "style": style,
            },
        }

    features = [as_timestamped(item) for item in items]

    yield TimestampedGeoJson(
        {"type": "FeatureCollection", "features": features}, duration="P10M"
    )


def visualize_items(items, style_function):
    def predicate(item):
        if "time" in item.properties and item.geometry["type"] == "LineString":
            type_ = "trajectory"
        else:
            type_ = "static"

        return type_

    grouped = groupby(predicate, items)
    transforms = {
        "static": as_geojson,
        "trajectory": as_timestamped_geojson,
    }

    for group, elems in grouped.items():
        transform = transforms.get(group)
        yield from transform(elems, style_function=style_function)

In [ ]:
def visualize_search(extent, items, m=None):
    if m is None:
        m = folium.Map(width="80%", height="80%", tiles="cartodbpositron")

    folium.GeoJson(extent.geojson()).add_to(m)
    for elem in visualize_items(items, style_function=style_function):
        elem.add_to(m)

    return m

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd

import shapely

from tlz.itertoolz import remove


def extract_trajectory(item):
    geometry = item.geometry
    coordinates = shapely.points(geometry["coordinates"])
    time = pd.to_datetime(item.properties["time"], format="ISO8601").astype(
        "datetime64[s]"
    )

    # TODO: find the crs info from the item properties
    gdf = gpd.GeoDataFrame({"time": time}, geometry=coordinates, crs=4326).set_index(
        "time"
    )
    return mpd.Trajectory(gdf, traj_id=item.id)


def filter_trajectories(items, geometry, timespan):
    def predicate(item):
        traj = extract_trajectory(item)

        segment = traj.get_segment_between(*pd.to_datetime(timespan))
        return not segment.intersects(geometry)

    return list(remove(predicate, items))

In [ ]:
geometry = BoundingBox(-15, 43, 5, 50).boundary()
timespan = ["2019-11-01", "2019-11-15"]

unfiltered = client.search(
    collections=["TS"],
    intersects=geometry,
    datetime=timespan,
    limit=100,
).item_collection()

filtered = filter_trajectories(unfiltered, geometry.geom, timespan)
len(unfiltered), len(filtered)

In [ ]:
m = folium.plugins.DualMap(layout="horizontal", tiles="cartodbpositron")
visualize_search(geometry, unfiltered, m=m.m1)
visualize_search(geometry, filtered, m=m.m2)

m